In [1]:
QLOG = 2252 * 1024
QCAM = QLOG
# QCAM = QCAM * 4 * 3  ( what if we increased qcam size and had three of them? )

RLOG = 56 * 1024 * 1024.0
DCAM = ECAM = FCAM = 73 * 1024 * 1024

In [2]:
TRAINING = RLOG + DCAM + ECAM + FCAM  # TODO: toggle dcam
DASHCAM = QLOG + QCAM

In [3]:
# comma 3X with OP 0.9.7
RESERVED_BYTES = (4.5 + 3.1 + 1.9) * (1024 ** 3)  # 4.5G staging, 3.1G openpilot, 1.9G scons cache
AVAILABLE_BYTES = 88 * (1024 ** 3) - RESERVED_BYTES  # 88G partition
# AVAILABLE_BYTES = 100 * 1024 * 1024 * 1024  # 100 GB

MIN_BYTES = 5 * 1024 * 1024 * 1024
MIN_PERCENT = 10

def calculate_used_space(routes_training: list[str], routes_dashcam: list[str]) -> int:
  return len(routes_training) * TRAINING + len(routes_dashcam) * DASHCAM

def get_available_percent(routes_training: list[str], routes_dashcam: list[str]) -> float:
  return 100 * (1 - calculate_used_space(routes_training, routes_dashcam) / AVAILABLE_BYTES)

def get_available_bytes(routes_training: list[str], routes_dashcam: list[str]) -> int:
  return AVAILABLE_BYTES - calculate_used_space(routes_training, routes_dashcam)

get_available_percent([], [])

100.0

In [4]:
class Logger:
  route_counter = 0

  def __init__(self):
    self.route_counter = 0

  def __next__(self) -> str:
    log_id = f"{self.route_counter:010}"
    self.route_counter += 1
    return log_id

next(Logger())

'0000000000'

In [5]:
def do_delete_original(routes_training: list[str], routes_dashcam: list[str]):
  # remove earliest route from both lists
  routes_training.pop(0)
  routes_dashcam.pop(0)

do_delete = do_delete_original

In [6]:
# class Deleter:
#   def __init__(self, minimum_bytes: int, minimum_percent: int, )

def deleter(routes_training: list[str], routes_dashcam: list[str]):
  while True:
    out_of_bytes = get_available_bytes(routes_training, routes_dashcam) < MIN_BYTES
    out_of_percent = get_available_percent(routes_training, routes_dashcam) < MIN_PERCENT

    if not (out_of_bytes or out_of_percent):
      return

    # TODO: avoid deleting most recent N preserved segments

    if len(routes_training) == 0 and len(routes_dashcam) == 0:
      return

    do_delete(routes_training, routes_dashcam)

In [7]:
import pandas as pd


def loop(count: int):
  logger = Logger()
  routes_training = []
  routes_dashcam = []

  header = ["segments_count", "training_count", "dashcam_count", "total_bytes", "training_bytes", "dashcam_bytes", "available_bytes", "available_percent"]
  rows = []

  for _ in range(count):
    # 1. log new segment
    log_id = next(logger)
    routes_training.append(log_id)
    routes_dashcam.append(log_id)

    # 2. run deleter
    deleter(routes_training, routes_dashcam)

    # 3. append stats
    training_count, dashcam_count = len(routes_training), len(routes_dashcam)
    segments_count = max(training_count, dashcam_count)
    rows.append([segments_count, training_count, dashcam_count, calculate_used_space(routes_training, routes_dashcam), training_count * TRAINING, dashcam_count * DASHCAM, get_available_bytes(routes_training, routes_dashcam), get_available_percent(routes_training, routes_dashcam)])

  return pd.DataFrame(rows, columns=header)


df = loop(1000)

,segments_count,training_count,dashcam_count,total_bytes,training_bytes,dashcam_bytes,available_bytes,available_percent
0,1,1,1,2.929705e+08,2.883584e+08,4612096,8.399576e+10,99.652420
1,2,2,2,5.859410e+08,5.767168e+08,9224192,8.370279e+10,99.304841
2,3,3,3,8.789115e+08,8.650752e+08,13836288,8.340982e+10,98.957261
3,4,4,4,1.171882e+09,1.153434e+09,18448384,8.311685e+10,98.609681
4,5,5,5,1.464852e+09,1.441792e+09,23060480,8.282388e+10,98.262102
...,...,...,...,...,...,...,...,...
995,258,258,258,7.558639e+10,7.439647e+10,1189920768,8.702345e+09,10.324447
996,258,258,258,7.558639e+10,7.439647e+10,1189920768,8.702345e+09,10.324447
997,258,258,258,7.558639e+10,7.439647e+10,1189920768,8.702345e+09,10.324447
998,258,258,258,7.558639e+10,7.439647e+10,1189920768,8.702345e+09,10.324447
